# FPP multi objective optimization

### Import packages

In [ ]:
#@time using WarmupFUSE
@time using FUSE
using Plots; gr();
FUSE.logging(Logging.Info; actors=Logging.Error);

### Define a working folder

Results will be save here

In [ ]:
save_folder = "optimization_run_LTSorHTS_ohtf3"

### Setup distributed computing environment

See more details here: https://fuse.help/parallel.html

In [ ]:
old_dir = pwd()
mkpath(save_folder)
try
    cd(save_folder) # this is to save temporary distributed files in working folder
    FUSE.parallel_environment("omega",128*2) # ("saga",120)
finally
    cd(old_dir)
end
display(pwd())
using Distributed
#@everywhere import WarmupFUSE
@everywhere import FUSE

### Get `ini` and `act` for FPP case and custmize as needed

In [ ]:
ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars, STEP=true)
act.ActorPFcoilsOpt.optimization_scheme = :none; # don't spend time optimizing the PFs
act.ActorStabilityLimits.models = [:beta_troyon_1984, :model_201, :model_401] # include βn check

### As a good practice, test the actor/workflow that you want to optimize

In [ ]:
#dd = FUSE.init(ini, act);
#FUSE.ActorWholeFacility(dd, act);

In [ ]:
#FUSE.digest(dd)

### Define optimization variables and ranges

In [ ]:
# nominal value and ranges
ini_opt = deepcopy(ini)
ini_opt.ec_launchers.power_launched = ini.ec_launchers.power_launched[1] ↔ [1e6, 200e6];
ini_opt.core_profiles.zeff = ini.core_profiles.zeff ↔ [1.1, 2.5]
#ini_opt.equilibrium.δ = ini.equilibrium.δ ↔ [-0.7,0.7]
#ini_opt.equilibrium.ζ = ini.equilibrium.ζ ↔ [0,0.2]
ini_opt.equilibrium.κ = missing # k set to be 95% of maximum controllable value
ini_opt.equilibrium.B0 = ini.equilibrium.B0 ↔ [1.0, 20.]
ini_opt.equilibrium.ip = ini.equilibrium.ip ↔ [1.0e6, 22e6]
ini_opt.equilibrium.R0 = ini.equilibrium.R0 ↔ [ini.equilibrium.R0, 10.0];

ini_opt.tf.technology = :HTS ↔ (:HTS, :LTS);
ini_opt.oh.technology = :HTS ↔ (:HTS, :LTS);

### See what are the possible optimization objectives and constraints

In [ ]:
# FUSE comes with a library of objective and constraints functions
OFL = deepcopy(FUSE.ObjectiveFunctionsLibrary)
CFL = deepcopy(FUSE.ConstraintFunctionsLibrary)
println("== OBJECTIVE FUNCTIONS ==")
display(OFL)
println()
println("== CONSTRAINT FUNCTIONS ==")
display(CFL)

## Set the optimization objectives and constraints

In [ ]:
objective_functions = [OFL[:min_βn], OFL[:min_capital_cost]]

# setup the constraint functions
CFL[:target_power_electric_net].limit = 200.0
CFL[:target_power_electric_net].tolerance = 0.01
constraint_functions = [CFL[:target_power_electric_net], CFL[:zero_ohmic]]

# ...but one can define custom objective/constraint functions too
# my_target_power_electric_net = FUSE.ObjectiveFunction(:my_target_power_electric_net, "MW", dd -> @ddtime(dd.balance_of_plant.power_electric_net)/1E6, 200.0)
# objective_functions = [my_target_power_electric_net, OFL[:min_βn], OFL[:min_capital_cost]]

println("== OBJECTIVE FUNCTIONS ==")
display(objective_functions)
println()
println("== CONSTRAINT FUNCTIONS ==")
display(constraint_functions)

### Setup and run optimization

In [ ]:
# option to resume an optimization where it was left off
if false
    continue_results = results
else
    continue_results = missing
end

# define optimization parameters
# For real optimization studies the population size (N) and number of iterations should be bigger
# eg. N=100, iterations=25
optimization_parameters = Dict(
    :N => max(4, Int(floor((nprocs()-1)/2))*2), # even number
    :iterations => 100,
    :continue_results => continue_results,
    :save_folder => save_folder)

# run optimization
results = FUSE.workflow_multiobjective_optimization(ini_opt, act, FUSE.ActorWholeFacility, objective_functions, constraint_functions; optimization_parameters...);

In [ ]:
import Metaheuristics
n_fes, fxs = Metaheuristics.convergence(results.state)

## Remember to always release computing resources!

In [ ]:
for i in workers()
    rmprocs(i)
end

### How to: Define and use a custom FUSE workflow

In [ ]:
# Here `@everywhere` is needed to make all processes aware of the custom function
@everywhere function workflow_custom(ini, act)
    FUSE.init(dd, ini, act)
    FUSE.ActorEquilibriumTransport(dd, act)
    FUSE.ActorCXbuild(dd, act)
    return dd
end

# results = FUSE.workflow_multiobjective_optimization(ini, act, custom_workflow, objective_functions; optimization_parameters...);